In [ ]:
import tiledb
from tiledb.cf import Group, GroupSchema, VirtualGroup
import numpy as np

In [ ]:
# Create a TileDB-CF GroupSchema
row = tiledb.Dim(name="rows", domain=(1, 4), tile=4, dtype=np.uint64)
col = tiledb.Dim(name="cols", domain=(1, 8), tile=8, dtype=np.uint64)
group_schema = GroupSchema(
    {
        "matrices": tiledb.ArraySchema(
            domain=tiledb.Domain(row, col), 
            attrs=[
                tiledb.Attr(name="A1", dtype=np.float64), 
                tiledb.Attr(name="A2", dtype=np.float64), 
                tiledb.Attr(name="A3", dtype=np.float64),
            ],
        ),
        "row_vectors": tiledb.ArraySchema(
            domain=tiledb.Domain(row), 
            attrs=[
                tiledb.Attr(name="y1", dtype=np.float64), 
                tiledb.Attr(name="y2", dtype=np.float64),                
            ],
        ),
        "column_vectors": tiledb.ArraySchema(
            domain=tiledb.Domain(col), 
            attrs=[
                tiledb.Attr(name="x1", dtype=np.float64), 
                tiledb.Attr(name="x2", dtype=np.float64), 
            ],
        ),
    }
)
group_schema.set_default_metadata_schema()
group_schema

In [ ]:
# Create a TileDB-CF Group
uri1 = "output/create_groups/example1"
uri2 = "output/create_groups/example2"
create_group = False
create_virtual_group = False
if create_group:
    Group.create(uri1, group_schema)
if create_virtual_group:
    Group.create(uri2, group_schema, is_virtual=True)

In [ ]:
# Check Example1 was created and try reading/writing
example1_schema = GroupSchema.load(uri1)
print(f"Schema is correct: {example1_schema == group_schema}")
with Group(uri1, attr="x1", mode="w") as group:
    group.array[:] = np.linspace(0.0, 1.0, 8)
    group.meta["dataset_name"] = "Example group dataset"
with Group(uri1) as group:
    print(f"Dataset name: {group.meta['dataset_name']}")
with Group(uri1, array="column_vectors") as group:
    data = group.array[:]
    print("Column vectors:")
    for attr_name, attr_data in data.items():
        print(f"  {attr_name}: {attr_data}")

In [ ]:
# Check Example2 was created and try reading/writing
#  * Since Example2 is a virtual group, we must create a dictionary
#    to the array URIs.
array_uris = {
    "__tiledb_group": uri2,
    "column_vectors": f"{uri2}_column_vectors",
    "row_vectors": f"{uri2}_row_vectors",
    "matrices": f"{uri2}_matrices",
}
example2_schema = GroupSchema.load_virtual(array_uris)
print(f"Schema is correct: {example2_schema == group_schema}") 
with VirtualGroup(array_uris, attr="x1", mode="w") as group:
    group.array[:] = np.linspace(0.0, 1.0, 8)
    group.meta["dataset_name"] = "Example group dataset"
with VirtualGroup(array_uris) as group:
    print(f"Dataset name: {group.meta['dataset_name']}")
with VirtualGroup(array_uris, array="column_vectors") as group:
    data = group.array[:]
    print("Column vectors:")
    for attr_name, attr_data in data.items():
        print(f"  {attr_name}: {attr_data}")